In [ ]:
import pandas as pd
from collections import Counter
import os
from glob import glob
import csv

The ATP dousbles CSV files had inconsistent formatting, including misaligned headers, extra commas, and irregular quoting. As a result, using column names with pandas oftern returned incorrect data. To ensure accurate extraction, I manually parsed each row and selected fields by fixed column positions based on the dataset's known structure. 

In [31]:
# These are the indices (column positions) based on the raw CSV row
column_indices = {
    'tourney_name': 1,
    'surface': 2,
    'round': 17,
    'tourney_date': 5,
    'winner1_name': 18,
    'winner2_name': 23,
    'loser1_name': 28,
    'loser2_name': 33,
    'winner1_ioc': 21,
    'winner2_ioc': 26,
    'loser1_ioc': 31,
    'loser2_ioc': 36,
    'winner1_age': 22,
    'winner2_age': 27,
    'loser1_age': 32,
    'loser2_age': 37,
    'score': 15,
    'minutes': 48
}

folder_path = 'data/doubles/'
file_paths = sorted(glob(os.path.join(folder_path, 'atp_matches_doubles_*.csv')))

records = []

for file_path in file_paths:
    year = int(os.path.basename(file_path).split('_')[-1].split('.')[0])

    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader)  # Skip header row

        for row in reader:
            try:
                record = {key: row[idx] if idx < len(row) else None for key, idx in column_indices.items()}
                record['year'] = year
                records.append(record)
            except Exception as e:
                print(f"Skipping broken row in {file_path}: {e}")
                continue

# Convert to DataFrame
df = pd.DataFrame(records)

# Drop rows with missing player names
required_names = ['winner1_name', 'winner2_name', 'loser1_name', 'loser2_name']
df.dropna(subset=required_names, inplace=True)

print("Done with manual loading and cleaning!")

Done with manual loading and cleaning!


Most doubles wins. This method ranks players by the total number of doubles matches they won, regardless of their partner. This reflects endurance, skill, and consistent success over time. 

In [32]:
# Count wins for each player in winner1_name and winner2_name
winners = pd.concat([df['winner1_name'], df['winner2_name']])
win_counts = winners.value_counts().reset_index()
win_counts.columns = ['player', 'wins']

print(win_counts.head(10))  # Top 10 most winning players


            player  wins
0       Mike Bryan  1111
1        Bob Bryan  1071
2    Daniel Nestor   846
3       Max Mirnyi   711
4   Nenad Zimonjic   675
5     Leander Paes   558
6  Mahesh Bhupathi   528
7     Mark Knowles   527
8     Marcelo Melo   514
9     Bruno Soares   480


Most finals won

In [33]:
final_winners = df[df['round'] == 'F']
finals = pd.concat([final_winners['winner1_name'], final_winners['winner2_name']])
final_win_counts = finals.value_counts().reset_index()
final_win_counts.columns = ['player', 'finals_won']

print(final_win_counts.head(10))


            player  finals_won
0       Mike Bryan         124
1        Bob Bryan         119
2    Daniel Nestor          78
3   Nenad Zimonjic          52
4       Max Mirnyi          47
5     Mark Knowles          43
6      Horia Tecau          37
7     Leander Paes          36
8  Mahesh Bhupathi          36
9   Jonas Bjorkman          34


best win ratio

In [38]:
# Count total wins per player
win_players = pd.concat([df['winner1_name'], df['winner2_name']])
win_counts = win_players.value_counts()

# Count total matches played (wins + losses)
all_players = pd.concat([
    df['winner1_name'], df['winner2_name'],
    df['loser1_name'], df['loser2_name']
])
match_counts = all_players.value_counts()

# Combine into a DataFrame for alignment
win_df = pd.DataFrame({
    'wins': win_counts,
    'matches': match_counts
}).dropna()

# Calculate win ratio
win_df['win_ratio'] = win_df['wins'] / win_df['matches']

# Filter by minimum match threshold
min_matches = 50
filtered = win_df[win_df['matches'] >= min_matches]

# Show top players by win ratio
filtered = filtered.sort_values('win_ratio', ascending=False).reset_index()
filtered.rename(columns={'index': 'player'}, inplace=True)

print(filtered[['player', 'win_ratio', 'matches', 'wins']].head(10))


                  player  win_ratio  matches    wins
0         Mark Woodforde   0.828125       64    53.0
1        Todd Woodbridge   0.771429      315   243.0
2              Bob Bryan   0.770504     1390  1071.0
3             Mike Bryan   0.769391     1444  1111.0
4          Paul Haarhuis   0.733871      124    91.0
5         Jonas Bjorkman   0.718261      575   413.0
6          Daniel Nestor   0.694581     1218   846.0
7           Mark Knowles   0.685306      769   527.0
8          Sandon Stolle   0.683962      212   145.0
9  Pierre Hugues Herbert   0.672340      235   158.0


most wins as a par - strongest team

In [36]:
# Sort pair names alphabetically to count consistently
df['team'] = df.apply(lambda row: tuple(sorted([row['winner1_name'], row['winner2_name']])), axis=1)
team_wins = df['team'].value_counts().reset_index()
team_wins.columns = ['team', 'wins']

print(team_wins.head(10))


                                      team  wins
0                  (Bob Bryan, Mike Bryan)  1071
1            (Daniel Nestor, Mark Knowles)   324
2  (Marcin Matkowski, Mariusz Fyrstenberg)   321
3     (Juan Sebastian Cabal, Robert Farah)   257
4          (Daniel Nestor, Nenad Zimonjic)   225
5              (Andy Ram, Jonathan Erlich)   223
6         (Horia Tecau, Jean Julien Rojer)   220
7             (Kevin Ullyett, Wayne Black)   176
8        (Jonas Bjorkman, Todd Woodbridge)   161
9           (Alexander Peya, Bruno Soares)   155


In [39]:
# Combine all players with surface info
player_surface = pd.concat([
    df[['winner1_name', 'surface']].rename(columns={'winner1_name': 'player'}),
    df[['winner2_name', 'surface']].rename(columns={'winner2_name': 'player'}),
    df[['loser1_name', 'surface']].rename(columns={'loser1_name': 'player'}),
    df[['loser2_name', 'surface']].rename(columns={'loser2_name': 'player'}),
])

player_surface.dropna(subset=['player', 'surface'], inplace=True)

# Count matches per player per surface
player_surface_counts = player_surface.groupby(['player', 'surface']).size().reset_index(name='matches')

print("Per Player Matches by Surface:")
print(player_surface_counts.head())


Per Player Matches by Surface:
                  player surface  matches
0                           Clay        1
1                          Grass        5
2                           Hard       11
3          Aaron Capitel    Hard        1
4  Abdul Hamid Makhkamov    Hard        1


In [40]:
# Create team columns (sort names so team order doesn't matter)
df['team'] = df.apply(lambda row: tuple(sorted([row['winner1_name'], row['winner2_name']])), axis=1)
df['surface'] = df['surface'].fillna('Unknown')

# Keep only relevant columns
team_surface = df[['team', 'surface']].dropna()

# Count how many matches each team played on each surface
team_surface_counts = team_surface.groupby(['team', 'surface']).size().reset_index(name='matches')

print("🎾 Per Team Matches by Surface:")
print(team_surface_counts.head())


🎾 Per Team Matches by Surface:
                              team surface  matches
0      (Adam Feeney, Carsten Ball)    Hard        3
1   (Adam Feeney, Colin Ebelthite)    Hard        2
2   (Adam Feeney, Johan Brunstrom)   Grass        2
3     (Adam Feeney, Robert Smeets)    Hard        1
4  (Adam Peterson, Alex Calatrava)    Clay        1
